In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import pickle
from src.modules.models import xgboost_reg, ensemble_feature_importance, regmodel_evaluation

## Load model

In [2]:
# load the model from disk
filename = 'output/xgb_model_012_noscale.pickle'

xgb_model = pickle.load(open(filename, 'rb'))
print(xgb_model)

{'XGBoost_Reg': XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.3, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=12, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)}


In [3]:
pd.set_option('display.max_columns', 100)
X_df = pd.read_csv('data/Test/test.csv')
print(X_df.shape)
X_df.head()

(150623, 83)


,mkt_unique_carrier,mkt_carrier_fl_num,tail_num,op_carrier_fl_num,origin_airport_id,origin,dest_airport_id,dest,crs_dep_time,crs_arr_time,crs_elapsed_time,distance,year,month,day,day_of_week,week_of_year,crs_dep_hour,crs_arr_hour,month_avg_arr_delay,month_avg_air_time,month_avg_carrier_delay,month_avg_weather_delay,month_avg_nas_delay,month_avg_security_delay,month_avg_late_aircraft_delay,day_of_week_avg_arr_delay,day_of_week_avg_air_time,day_of_week_avg_carrier_delay,day_of_week_avg_weather_delay,day_of_week_avg_nas_delay,day_of_week_avg_security_delay,day_of_week_avg_late_aircraft_delay,arr_hour_avg_arr_delay,arr_hour_avg_air_time,arr_hour_avg_carrier_delay,arr_hour_avg_weather_delay,arr_hour_avg_nas_delay,arr_hour_avg_security_delay,arr_hour_avg_late_aircraft_delay,origin_total_flights,dest_total_flights,month_flight_avg_seats,month_flight_avg_passengers,carrier_month_seats,carrier_month_passengers,carrier_month_avg_seats,carrier_month_avg_passengers,month_avg_fuel_comsumption,orig_airport_month_flight_seats,orig_airport_month_passengers,dest_airport_month_flight_seats,dest_airport_month_passengers,origin_min_temp,origin_max_temp,origin_avg_temp,origin_total_snow_cm,origin_windspeed_kmph,origin_precipitation_mm,origin_humidity,origin_visibility,origin_cloud_cover,origin_heat_index_c,origin_wind_chill_c,origin_wind_gust_kmph,origin_feels_like,origin_weather_description,origin_pressure,dest_min_temp,dest_max_temp,dest_avg_temp,dest_total_snow_cm,dest_windspeed_kmph,dest_precipitation_mm,dest_humidity,dest_visibility,dest_cloud_cover,dest_heat_index_c,dest_wind_chill_c,dest_wind_gust_kmph,dest_feels_like,dest_weather_description,dest_pressure
0,WN,5888,N951WN,5888,13891,ONT,14771,SFO,1810,1945,95,363,2020,1,1,3,1,18,19,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,3.125000,57.125000,1.478261,0.00,42.521739,0.0,9.956522,5.258824,56.623529,18.421053,0.0,25.368421,0.0,26.473684,7086.0,57334.0,152.074203,113.048765,79269676,58102809,154.052359,113.739038,156532592.0,2303449.0,1741130.0,24365383.0,18967854.0,10.0,18.0,15.0,0.0,4.0,0.0,34.0,10.0,5.0,14.0,14.0,7.0,14.0,Partly cloudy,1017.0,11.0,14.0,13.0,0.0,12.0,0.0,83.0,10.0,29.0,13.0,12.0,17.0,12.0,Partly cloudy,1021.0
1,WN,6276,N467WN,6276,13891,ONT,14771,SFO,1150,1320,90,363,2020,1,1,3,1,11,13,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,3.125000,57.125000,1.478261,0.00,42.521739,0.0,9.956522,21.142857,59.142857,0.000000,0.0,17.500000,0.0,77.000000,7086.0,57334.0,152.074203,113.048765,79269676,58102809,154.052359,113.739038,156532592.0,2303449.0,1741130.0,24365383.0,18967854.0,10.0,18.0,15.0,0.0,4.0,0.0,34.0,10.0,5.0,14.0,14.0,7.0,14.0,Partly cloudy,1017.0,11.0,14.0,13.0,0.0,12.0,0.0,83.0,10.0,29.0,13.0,12.0,17.0,12.0,Partly cloudy,1021.0
2,WN,4598,N7885A,4598,13891,ONT,14831,SJC,2020,2130,70,333,2020,1,1,3,1,20,21,-0.467742,53.842742,9.846154,0.0,7.128205,1.948718,20.102564,-1.183761,52.297009,9.200000,0.72,3.920000,0.0,33.140000,8.811881,51.735974,7.871429,0.0,2.728571,0.0,47.800000,7086.0,20962.0,155.201183,98.481989,79269676,58102809,154.052359,113.739038,156532592.0,2303449.0,1741130.0,6173045.0,4355999.0,10.0,18.0,15.0,0.0,4.0,0.0,34.0,10.0,5.0,14.0,14.0,7.0,14.0,Partly cloudy,1017.0,8.0,16.0,12.0,0.0,6.0,0.0,71.0,10.0,44.0,11.0,11.0,8.0,11.0,Overcast,1021.0
3,WN,4761,N551WN,4761,13891,ONT,14831,SJC,1340,1455,75,333,2020,1,1,3,1,13,14,-0.467742,53.842742,9.846154,0.0,7.128205,1.948718,20.102564,-1.183761,52.297009,9.200000,0.72,3.920000,0.0,33.140000,2.074074,51.700000,15.717391,0.0,2.239130,0.0,21.956522,7086.0,20962.0,155.201183,98.481989,79269676,58102809,154.052359,113.739038,156532592.0,2303449.0,1741130.0,6173045.0,4355999.0,10.0,18.0,15.0,0.0,4.0,0.0,34.0,10.0,5.0,14.0,14.0,7.0,14.0,Partly cloudy,1017.0,8.0,16.0,12.0,0.0,6.0,0.0,71.0,10.0,44.0,11.0,11.0,8.0,11.0,Overcast,1021.0
4,WN,5162,N968WN,5162,13891,ONT,14831,SJC,915,1035,80,333,2020,1,1,3,1,9,10,-0.467742,53.842742,9.846154,0.0,7.128205,1.948718,20.102564,-1.183761,52.297009,9.200000,

In [4]:

X_df = X_df.loc[:, ['mkt_unique_carrier', 'crs_elapsed_time', 'year', 'month', 'day', 'day_of_week', 'crs_dep_hour', 'crs_arr_hour', 'month_avg_arr_delay',
             'month_avg_carrier_delay', 'month_avg_weather_delay', 'month_avg_nas_delay', 'month_avg_security_delay', 'month_avg_late_aircraft_delay', 
             'day_of_week_avg_arr_delay', 'day_of_week_avg_carrier_delay', 'day_of_week_avg_weather_delay', 'day_of_week_avg_nas_delay', 
             'day_of_week_avg_security_delay', 'day_of_week_avg_late_aircraft_delay', 'arr_hour_avg_arr_delay', 'arr_hour_avg_carrier_delay', 
             'arr_hour_avg_weather_delay', 'arr_hour_avg_nas_delay', 'arr_hour_avg_security_delay', 'arr_hour_avg_late_aircraft_delay', 'origin_total_flights', 
             'dest_total_flights', 'month_flight_avg_seats', 'carrier_month_seats', 'carrier_month_avg_seats', 'origin_min_temp', 'origin_total_snow_cm', 
             'origin_windspeed_kmph', 'origin_precipitation_mm', 'origin_humidity', 'origin_visibility', 'origin_cloud_cover', 'origin_pressure', 'dest_min_temp', 
             'dest_total_snow_cm', 'dest_windspeed_kmph', 'dest_precipitation_mm', 'dest_humidity', 'dest_visibility', 'dest_cloud_cover', 'dest_pressure']]
print(X_df.shape)
X_df.head()

(150623, 47)


,mkt_unique_carrier,crs_elapsed_time,year,month,day,day_of_week,crs_dep_hour,crs_arr_hour,month_avg_arr_delay,month_avg_carrier_delay,month_avg_weather_delay,month_avg_nas_delay,month_avg_security_delay,month_avg_late_aircraft_delay,day_of_week_avg_arr_delay,day_of_week_avg_carrier_delay,day_of_week_avg_weather_delay,day_of_week_avg_nas_delay,day_of_week_avg_security_delay,day_of_week_avg_late_aircraft_delay,arr_hour_avg_arr_delay,arr_hour_avg_carrier_delay,arr_hour_avg_weather_delay,arr_hour_avg_nas_delay,arr_hour_avg_security_delay,arr_hour_avg_late_aircraft_delay,origin_total_flights,dest_total_flights,month_flight_avg_seats,carrier_month_seats,carrier_month_avg_seats,origin_min_temp,origin_total_snow_cm,origin_windspeed_kmph,origin_precipitation_mm,origin_humidity,origin_visibility,origin_cloud_cover,origin_pressure,dest_min_temp,dest_total_snow_cm,dest_windspeed_kmph,dest_precipitation_mm,dest_humidity,dest_visibility,dest_cloud_cover,dest_pressure
0,WN,95,2020,1,1,3,18,19,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,3.125000,1.478261,0.00,42.521739,0.0,9.956522,5.258824,18.421053,0.0,25.368421,0.0,26.473684,7086.0,57334.0,152.074203,79269676,154.052359,10.0,0.0,4.0,0.0,34.0,10.0,5.0,1017.0,11.0,0.0,12.0,0.0,83.0,10.0,29.0,1021.0
1,WN,90,2020,1,1,3,11,13,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,3.125000,1.478261,0.00,42.521739,0.0,9.956522,21.142857,0.000000,0.0,17.500000,0.0,77.000000,7086.0,57334.0,152.074203,79269676,154.052359,10.0,0.0,4.0,0.0,34.0,10.0,5.0,1017.0,11.0,0.0,12.0,0.0,83.0,10.0,29.0,1021.0
2,WN,70,2020,1,1,3,20,21,-0.467742,9.846154,0.0,7.128205,1.948718,20.102564,-1.183761,9.200000,0.72,3.920000,0.0,33.140000,8.811881,7.871429,0.0,2.728571,0.0,47.800000,7086.0,20962.0,155.201183,79269676,154.052359,10.0,0.0,4.0,0.0,34.0,10.0,5.0,1017.0,8.0,0.0,6.0,0.0,71.0,10.0,44.0,1021.0
3,WN,75,2020,1,1,3,13,14,-0.467742,9.846154,0.0,7.128205,1.948718,20.102564,-1.183761,9.200000,0.72,3.920000,0.0,33.140000,2.074074,15.717391,0.0,2.239130,0.0,21.956522,7086.0,20962.0,155.201183,79269676,154.052359,10.0,0.0,4.0,0.0,34.0,10.0,5.0,1017.0,8.0,0.0,6.0,0.0,71.0,10.0,44.0,1021.0
4,WN,80,2020,1,1,3,9,10,-0.467742,9.846154,0.0,7.128205,1.948718,20.102564,-1.183761,9.200000,0.72,3.920000,0.0,33.140000,-2.740260,75.000000,0.0,17.833333,0.0,13.000000,7086.0,20962.0,155.201183,79269676,154.052359,10.0,0.0,4.0,0.0,34.0,10.0,5.0,1017.0,8.0,0.0,6.0,0.0,71.0,10.0,44.0,1021.0


## Feature scale and encode

In [5]:
from src.modules.feature_engineering import encoder

In [6]:
scaled_df = encoder(X_df, drop_first=False)
print(scaled_df.shape)
scaled_df.head()

(150623, 56)


,crs_elapsed_time,year,month,day,day_of_week,crs_dep_hour,crs_arr_hour,month_avg_arr_delay,month_avg_carrier_delay,month_avg_weather_delay,month_avg_nas_delay,month_avg_security_delay,month_avg_late_aircraft_delay,day_of_week_avg_arr_delay,day_of_week_avg_carrier_delay,day_of_week_avg_weather_delay,day_of_week_avg_nas_delay,day_of_week_avg_security_delay,day_of_week_avg_late_aircraft_delay,arr_hour_avg_arr_delay,arr_hour_avg_carrier_delay,arr_hour_avg_weather_delay,arr_hour_avg_nas_delay,arr_hour_avg_security_delay,arr_hour_avg_late_aircraft_delay,origin_total_flights,dest_total_flights,month_flight_avg_seats,carrier_month_seats,carrier_month_avg_seats,origin_min_temp,origin_total_snow_cm,origin_windspeed_kmph,origin_precipitation_mm,origin_humidity,origin_visibility,origin_cloud_cover,origin_pressure,dest_min_temp,dest_total_snow_cm,dest_windspeed_kmph,dest_precipitation_mm,dest_humidity,dest_visibility,dest_cloud_cover,dest_pressure,mkt_unique_carrier_AA,mkt_unique_carrier_AS,mkt_unique_carrier_B6,mkt_unique_carrier_DL,mkt_unique_carrier_F9,mkt_unique_carrier_G4,mkt_unique_carrier_HA,mkt_unique_carrier_NK,mkt_unique_carrier_UA,mkt_unique_carrier_WN
0,95,2020,1,1,3,18,19,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,3.125000,1.478261,0.00,42.521739,0.0,9.956522,5.258824,18.421053,0.0,25.368421,0.0,26.473684,7086.0,57334.0,152.074203,79269676,154.052359,10.0,0.0,4.0,0.0,34.0,10.0,5.0,1017.0,11.0,0.0,12.0,0.0,83.0,10.0,29.0,1021.0,0,0,0,0,0,0,0,0,0,1
1,90,2020,1,1,3,11,13,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,3.125000,1.478261,0.00,42.521739,0.0,9.956522,21.142857,0.000000,0.0,17.500000,0.0,77.000000,7086.0,57334.0,152.074203,79269676,154.052359,10.0,0.0,4.0,0.0,34.0,10.0,5.0,1017.0,11.0,0.0,12.0,0.0,83.0,10.0,29.0,1021.0,0,0,0,0,0,0,0,0,0,1
2,70,2020,1,1,3,20,21,-0.467742,9.846154,0.0,7.128205,1.948718,20.102564,-1.183761,9.200000,0.72,3.920000,0.0,33.140000,8.811881,7.871429,0.0,2.728571,0.0,47.800000,7086.0,20962.0,155.201183,79269676,154.052359,10.0,0.0,4.0,0.0,34.0,10.0,5.0,1017.0,8.0,0.0,6.0,0.0,71.0,10.0,44.0,1021.0,0,0,0,0,0,0,0,0,0,1
3,75,2020,1,1,3,13,14,-0.467742,9.846154,0.0,7.128205,1.948718,20.102564,-1.183761,9.200000,0.72,3.920000,0.0,33.140000,2.074074,15.717391,0.0,2.239130,0.0,21.956522,7086.0,20962.0,155.201183,79269676,154.052359,10.0,0.0,4.0,0.0,34.0,10.0,5.0,1017.0,8.0,0.0,6.0,0.0,71.0,10.0,44.0,1021.0,0,0,0,0,0,0,0,0,0,1
4,80,2020,1,1,3,9,10,-0.467742,9.846154,0.0,7.128205,1.948718,20.102564,-1.183761,9.200000,0.72,3.920000,0.0,33.140000,-2.740260,75.000000,0.0,17.833333,0.0,13.000000,7086.0,20962.0,155.201183,79269676,154.052359,10.0,0.0,4.0,0.0,34.0,10.0,5.0,1017.0,8.0,0.0,6.0,0.0,71.0,10.0,44.0,1021.0,0,0,0,0,0,0,0,0,0,1


In [7]:
from src.modules.models import regmodel_evaluation

In [8]:
X = scaled_df.values
X.shape

(150623, 56)

In [9]:
arr_delay_predictions = xgb_model['XGBoost_Reg'].predict(X)
print(arr_delay_predictions[:50])

[323.52313 312.78128 340.47223 337.21356 330.76984 302.32352 340.21472
 327.15256 317.80765 312.83606 320.70883 330.41116 325.78235 328.93198
 355.02084 349.0824  364.72318 361.48593 357.27847 351.95605 303.77042
 271.22388 311.30692 299.08582 319.19064 351.40097 323.3974  324.045
 340.08075 349.42566 349.44705 320.34793 307.20114 312.37497 304.5241
 296.65134 355.00244 318.07013 323.92868 352.9694  314.48828 324.2887
 320.0651  314.9224  280.08676 349.93085 311.3316  330.1739  331.64542
 327.6034 ]


In [10]:
arr_delay_predictions.mean()

337.95517

In [11]:
arr_delay_predictions.max()

431.61887

In [12]:
arr_delay_predictions.min()

176.35841

In [13]:
np.median(arr_delay_predictions)

340.99054

## Create submit csv file

In [14]:
# Read test fields to join to predictions
submit_test = pd.read_csv('data/Test/submit_test.csv')
submit_test.shape

(150623, 5)

In [15]:
# Create predictions DataFrame to join with submit_test
predictions = pd.DataFrame(arr_delay_predictions, columns=['predicted_delay'])
predictions.shape

(150623, 1)

In [23]:
submit_final = pd.merge(submit_test, predictions, how='left' , left_index=True, right_index=True)
print(submit_final.shape)
submit_final.head()

(150623, 6)


,fl_date,mkt_carrier,mkt_carrier_fl_num,origin,dest,predicted_delay
0,2020-01-01,WN,5888,ONT,SFO,323.523132
1,2020-01-01,WN,6276,ONT,SFO,312.781281
2,2020-01-01,WN,4598,ONT,SJC,340.472229
3,2020-01-01,WN,4761,ONT,SJC,337.213562
4,2020-01-01,WN,5162,ONT,SJC,330.769836


In [22]:
submit_final.to_csv('data/Submission/Flight_Delay_Predictor.csv', index=False)

# Load Second model


XGBoost_Reg metrics:
	R2 = 0.11557689644765556
	Adjusted R2 = 0.11505094414178874
	RMSE = 45.02327820999893

*******************************************************************

In [25]:
# load the model from disk
filename = 'output/XGboost_model_20200702.pickle'

xgb_model_2 = pickle.load(open(filename, 'rb'))
print(xgb_model_2)

{'XGBoost_Reg': XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=100, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.1, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=500, n_jobs=4, num_parallel_tree=1, random_state=0,
             reg_alpha=0.1, reg_lambda=2.5, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)}


In [26]:
pd.set_option('display.max_columns', 100)
X_2 = pd.read_csv('data/Test/test_cleaned_XGboost_model_20200702.csv')
print(X_df.shape)
X_2.head()

(150623, 69)


,origin_airport_id,dest_airport_id,crs_dep_time,crs_arr_time,crs_elapsed_time,distance,year,month,day,day_of_week,week_of_year,crs_dep_hour,crs_arr_hour,month_avg_arr_delay,month_avg_air_time,month_avg_carrier_delay,month_avg_weather_delay,month_avg_nas_delay,month_avg_security_delay,month_avg_late_aircraft_delay,day_of_week_avg_arr_delay,day_of_week_avg_air_time,day_of_week_avg_carrier_delay,day_of_week_avg_weather_delay,day_of_week_avg_nas_delay,day_of_week_avg_security_delay,day_of_week_avg_late_aircraft_delay,arr_hour_avg_arr_delay,arr_hour_avg_air_time,arr_hour_avg_carrier_delay,arr_hour_avg_weather_delay,arr_hour_avg_nas_delay,arr_hour_avg_security_delay,arr_hour_avg_late_aircraft_delay,origin_total_flights,dest_total_flights,month_flight_avg_passengers,carrier_month_avg_passengers,month_avg_fuel_comsumption,origin_avg_temp,origin_total_snow_cm,origin_windspeed_kmph,origin_precipitation_mm,origin_humidity,origin_visibility,origin_wind_gust_kmph,origin_pressure,dest_avg_temp,dest_total_snow_cm,dest_windspeed_kmph,dest_precipitation_mm,dest_humidity,dest_visibility,dest_wind_gust_kmph,dest_pressure,orig_airport_capacity_month,dest_airport_capacity_month,tail_num_n,mkt_unique_carrier_AA,mkt_unique_carrier_AS,mkt_unique_carrier_B6,mkt_unique_carrier_DL,mkt_unique_carrier_F9,mkt_unique_carrier_G4,mkt_unique_carrier_HA,mkt_unique_carrier_NK,mkt_unique_carrier_UA,mkt_unique_carrier_VX,mkt_unique_carrier_WN
0,0.78151,1.359594,0.963882,0.876848,-0.660447,-0.717995,0.0,0.0,-1.528764,0.022398,-0.638796,1.002153,0.847341,-0.407247,-1.491645,-1.151244,-0.446521,-1.10933,-0.103027,-1.517865,-0.283842,-0.757808,-1.177243,-0.515554,2.439603,-0.140369,-1.128269,-0.011252,-0.750790,-0.172918,-0.407892,0.708671,-0.134473,0.107054,-1.041543,0.355951,-6.890758e-14,-0.578069,0.618029,0.865879,-0.142856,-1.317428,-0.35558,-1.339583,0.442484,-1.229544,0.206412,0.596879,-0.14268,-0.095306,-0.355458,1.024962,0.442559,-0.167188,0.226159,-0.000341,0.417302,-0.028654,-0.600944,-0.246486,-0.19605,-0.501487,-0.142361,-0.120308,-0.11044,-0.167257,-0.499496,0,2.238467
1,0.78151,1.359594,-0.372765,-0.315885,-0.728836,-0.717995,0.0,0.0,-1.528764,0.022398,-0.638796,-0.420511,-0.298153,-0.407247,-1.491645,-1.151244,-0.446521,-1.10933,-0.103027,-1.517865,-0.283842,-0.757808,-1.177243,-0.515554,2.439603,-0.140369,-1.128269,1.370303,-0.715039,-0.942015,-0.407892,0.186810,-0.134473,2.639065,-1.041543,0.355951,-6.890758e-14,-0.578069,0.618029,0.865879,-0.142856,-1.317428,-0.35558,-1.339583,0.442484,-1.229544,0.206412,0.596879,-0.14268,-0.095306,-0.355458,1.024962,0.442559,-0.167188,0.226159,-0.000341,0.417302,-0.028656,-0.600944,-0.246486,-0.19605,-0.501487,-0.142361,-0.120308,-0.11044,-0.167257,-0.499496,0,2.238467
2,0.78151,1.398996,1.389179,1.229897,-1.002392,-0.768505,0.0,0.0,-1.528764,0.022398,-0.638796,1.408629,1.229172,-0.455558,-0.744025,-0.591227,-0.446521,-0.55462,2.174326,-0.261819,-0.802586,-0.826856,-0.709641,-0.416913,-0.971386,-0.140369,0.444889,0.297785,-0.820147,-0.613375,-0.407892,-0.792880,-0.134473,1.175774,-1.041543,-0.655865,-5.351017e-01,-0.578069,0.618029,0.865879,-0.142856,-1.317428,-0.35558,-1.339583,0.442484,-1.229544,0.206412,0.461947,-0.14268,-1.012454,-0.355458,0.445747,0.442559,-1.124024,0.226159,-0.000341,-0.927852,-0.028899,-0.600944,-0.246486,-0.19605,-0.501487,-0.142361,-0.120308,-0.11044,-0.167257,-0.499496,0,2.238467
3,0.78151,1.398996,0.012027,-0.058255,-0.934003,-0.768505,0.0,0.0,-1.528764,0.022398,-0.638796,-0.014036,-0.107237,-0.455558,-0.744025,-0.591227,-0.446521,-0.55462,2.174326,-0.261819,-0.802586,-0.826856,-0.709641,-0.416913,-0.971386,-0.140369,0.444889,-0.288254,-0.820658,-0.285798,-0.407892,-0.825342,-0.134473,-0.119313,-1.041543,-0.655865,-5.351017e-01,-0.578069,0.618029,0.865879,-0.142856,-1.317428,-0.35558,-1.339583,0.442484,-1.229544,0.206412,0.461947,-0.14268,-1.012454,-0.355458,0.445747,0.442559,-1.124024,0.226159,-0.000341,-0.927852,-0.028655,-0.600944,-0.246486,-0.19605,-0.501487,-0.142361,-0.120308,-0.11

In [28]:
arr_delay_pred_2 = xgb_model_2['XGBoost_Reg'].predict(X_2)
print(arr_delay_pred_2[:50])

[  9.75589      7.5752883    2.0875394   -1.9019548   -6.5508914
  -7.2362347    0.32463366  -1.5952151   -4.727334    -6.2911434
  -5.6865144    2.2848234    8.127962    -1.6528575    0.2518154
  -2.2528558    0.4993619    4.001479   -11.051626     1.9690094
  -2.596329   -12.498283    -6.3675656  -10.341663    -5.923789
  12.004529    14.56224      0.86026067  -5.5119734    4.693937
   8.631359    11.015806    -7.5117908    5.2255483    2.9849632
  -6.404431    10.350584    -5.2691307   -6.817314    -1.6241322
  -9.575067     5.4478335    8.936757    -6.3992896   -8.850795
  -4.61195     -9.428158     1.0295984   -8.840016     2.6672888 ]
